In [2]:
from PIL import Image
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score,f1_score,classification_report
import os
import gc

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
!pip install gdown

In [4]:
import gdown
gdown.download("https://drive.google.com/uc?id=1wqcODyaPE3AzGaWVVRtY9jsM0sT_6Ecn","best_model.pth",quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1wqcODyaPE3AzGaWVVRtY9jsM0sT_6Ecn
From (redirected): https://drive.google.com/uc?id=1wqcODyaPE3AzGaWVVRtY9jsM0sT_6Ecn&confirm=t&uuid=5f2801a1-f92f-4c08-b1b7-08c8becdd419
To: /kaggle/working/best_model.pth
100%|██████████| 620M/620M [00:04<00:00, 152MB/s]  


'best_model.pth'

In [5]:
gdown.download("https://drive.google.com/uc?id=1bfgsVn_MvMkL5g_0wfANV3uj2-2sG8uf","data500x50.zip",quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1bfgsVn_MvMkL5g_0wfANV3uj2-2sG8uf
From (redirected): https://drive.google.com/uc?id=1bfgsVn_MvMkL5g_0wfANV3uj2-2sG8uf&confirm=t&uuid=224fc18b-31a7-4452-bb6b-cb0dc27fb6c3
To: /kaggle/working/data500x50.zip
100%|██████████| 2.08G/2.08G [00:17<00:00, 118MB/s] 


'data500x50.zip'

In [6]:
!unzip /kaggle/working/data500x50.zip

Archive:  /kaggle/working/data500x50.zip
  inflating: data500x50/train_data.npy  
  inflating: data500x50/train_label.npy  
  inflating: data500x50/val_data.npy  
  inflating: data500x50/val_label.npy  


In [ ]:
class model(nn.Module):
    def __init__(self):
        super(model,self).__init__()
        self.feature_ext = nn.Sequential(
            nn.Conv2d(1,5,5,padding=2),
            nn.ReLU(),
            nn.Conv2d(5,10,5,padding = 2),
            nn.MaxPool2d(2,stride=2),
            nn.Conv2d(10,15,3,padding=1),
            nn.ReLU(),
            nn.Conv2d(15,20,3,padding = 1),
            nn.MaxPool2d(2,stride=2),
            nn.Flatten(),
            nn.Linear(30000,5000),
            nn.Sigmoid(),
            nn.Linear(5000,1000),
        )
        self.out = nn.Sequential(
            nn.Linear(1,2),
            nn.Sigmoid(),
            nn.Softmax(dim = 1)
        )
    
    def forward(self,X1,X2):
        feature1 = self.feature_ext(X1)
        feature2 = self.feature_ext(X2)
        diff = feature1 - feature2
        Z = torch.norm(diff,dim = 1).reshape((-1,1))
        return self.out(Z)

In [7]:
best_auc = 0

In [7]:
X = np.load("data500x50/train_data.npy")
Y = np.load("data500x50/train_label.npy")
X_v = np.load("data500x50/val_data.npy")
Y_v = np.load("data500x50/val_label.npy")

In [8]:
num_batch_train,_,batch_size_train,_,_,_ = X.shape
num_batch_val,_,batch_size_val,_,_,_ = X_v.shape

In [ ]:
class model_2(nn.Module):
    def __init__(self):
        super(model_2,self).__init__()
        self.feature_ext = nn.Sequential(
            nn.Conv2d(1,5,5,padding=2),
            nn.ReLU(),
            nn.Conv2d(5,10,5,padding = 2),
            nn.MaxPool2d(2,stride=2),
            nn.Conv2d(10,15,3,padding=1),
            nn.ReLU(),
            nn.Conv2d(15,20,3,padding = 1),
            nn.MaxPool2d(2,stride=2),
            nn.Flatten(),
            nn.Linear(30000,5000),
            nn.Sigmoid(),
            nn.Linear(5000,1000),
        )
        self.out = nn.Sequential(
            nn.Linear(1000,50),
            nn.Sigmoid(),
            nn.Linear(50,2),
            nn.Softmax(dim = 1)
        )
    
    def forward(self,X1,X2):
        feature1 = self.feature_ext(X1)
        feature2 = self.feature_ext(X2)
        diff = feature1 - feature2
        diff = torch.abs(diff)
        return self.out(diff)

In [9]:
class model_3(nn.Module):
    def __init__(self):
        super(model_3,self).__init__()
        self.feature_ext = nn.Sequential(
            nn.Conv2d(1,5,5,padding=2),
            nn.ReLU(),
            nn.Conv2d(5,10,5,padding = 2),
            nn.MaxPool2d(2,stride=2),
            nn.Conv2d(10,15,3,padding=1),
            nn.ReLU(),
            nn.Conv2d(15,20,3,padding = 1),
            nn.MaxPool2d(2,stride=2),
            nn.Flatten(),
            nn.Linear(30000,5000),
            nn.ReLU(),
            nn.Linear(5000,1000),
        )
        self.out = nn.Sequential(
            nn.Linear(1000,50),
            nn.ReLU(),
            nn.Linear(50,2),
            nn.Softmax(dim = 1)
        )
    
    def forward(self,X1,X2):
        feature1 = self.feature_ext(X1)
        feature2 = self.feature_ext(X2)
        diff = feature1 - feature2
        diff = torch.abs(diff)
        return self.out(diff)

In [ ]:
gdown.download('https://drive.google.com/uc?id=17wa93OwnIb-z6NlK7-TjBYopLVBFkugq','best_model.pth',quiet=False)

In [10]:
mod = torch.load("best_model.pth")

In [10]:
best_auc = 0.0
best_f1 = 0.0

In [19]:
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(mod.parameters(),lr = 0.0001)
pred_Y = np.zeros(num_batch_val*batch_size_val)
mod = mod.to('cuda:0')
for epoch in range(2):
    total_loss = 0
    print(f"Epoch {epoch}")
    for i in range(num_batch_train):
        X1 = torch.tensor(X[i,0]).float()
        X2 = torch.tensor(X[i,1]).float()
        Y_ = torch.tensor(Y[i])
        X1 = X1.to('cuda:0')
        X2 = X2.to('cuda:0')
        Y_ = Y_.to('cuda:0')
        opt.zero_grad()
        prob = mod(X1.float(),X2.float())
        del X2
        del X1
        loss = loss_fn(prob,Y_.long())
        total_loss += loss.item()
        loss.backward()

        opt.step()
        
        del loss
        del prob
    print(f"Training \t loss : {total_loss/num_batch_train}")
    with torch.no_grad():
        for i in range(num_batch_val):
            X1 = torch.tensor(X_v[i,0]).float()
            X2 = torch.tensor(X_v[i,1]).float()
            X1 = X1.to('cuda:0')
            X2 = X2.to('cuda:0')
            prob = mod(X1.float(),X2.float())
            pred_Y[i*batch_size_val:(i+1)*batch_size_val] = prob.cpu().numpy()[:,1]
            del prob
            del X2
            del X1
    print(f"Validation \t F1 : {f1_score(Y_v,(pred_Y>=0.5)*1)} \t AUC : {roc_auc_score(Y_v,pred_Y)}")
    if best_auc < roc_auc_score(Y_v,pred_Y):
        best_auc = roc_auc_score(Y_v,pred_Y)
        torch.save(mod,'best_auc_model.pth')
    gc.collect()
    torch.cuda.empty_cache()

Epoch 0
Training 	 loss : 0.4998748333863639
Validation 	 F1 : 0.7576075550891919 	 AUC : 0.8391095398718645
Epoch 1


KeyboardInterrupt: 

In [14]:
print(classification_report(Y_v,(pred_Y>=0.5)*1))

              precision    recall  f1-score   support

           0       0.79      0.88      0.83      2885
           1       0.86      0.76      0.81      2885

    accuracy                           0.82      5770
   macro avg       0.83      0.82      0.82      5770
weighted avg       0.83      0.82      0.82      5770



In [ ]:
df = pd.DataFrame()

In [ ]:
df['label'] = (pred_Y>=0.5)*1
df

In [ ]:
df['proba'] = pred_Y
df

In [ ]:
df.to_csv("pred.csv")

In [ ]:
from IPython.display import FileLink
FileLink(r'best_auc_model.pth')

In [12]:
train_X = np.zeros((num_batch_train*batch_size_train,1000))
train_Y = np.zeros((num_batch_train*batch_size_train,1))
val_X = np.zeros((num_batch_val*batch_size_val,1000))
val_Y = np.zeros((num_batch_val*batch_size_val,1))
mod.to('cuda:0')
with torch.no_grad():
    for i in range(num_batch_train):
        X1 = torch.tensor(X[i,0]).float()
        X2 = torch.tensor(X[i,1]).float()
        X1 = X1.to('cuda:0')
        X2 = X2.to('cuda:0')
        feature1 = mod.feature_ext(X1)
        feature2 = mod.feature_ext(X2)
        diff = feature1 - feature2
        diff = torch.abs(diff)
        train_X[i*batch_size_train:(i+1)*batch_size_train,:] = diff.cpu().numpy()
        train_Y[i*batch_size_train:(i+1)*batch_size_train] = Y[i].reshape((-1,1))
    for i in range(num_batch_val):
        X1 = torch.tensor(X_v[i,0]).float()
        X2 = torch.tensor(X_v[i,1]).float()
        X1 = X1.to('cuda:0')
        X2 = X2.to('cuda:0')
        feature1 = mod.feature_ext(X1)
        feature2 = mod.feature_ext(X2)
        diff = feature1 - feature2
        diff = torch.abs(diff)
        val_X[i*batch_size_val:(i+1)*batch_size_val,:] = diff.cpu().numpy()
        val_Y[i*batch_size_val:(i+1)*batch_size_val] = Y_v[i]
    

In [19]:
val_Y = Y_v.reshape((-1,1))

In [20]:
val_Y

array([[1],
       [0],
       [1],
       ...,
       [1],
       [1],
       [0]])

In [24]:
with open("val_X.npy",'wb') as f:
    np.save(f,val_X)

In [25]:
!zip -v dataset100.zip train_X.npy train_Y.npy val_X.npy val_Y.npy

  adding: train_X.npy ........................................................................	(in=759200128) (out=378888435) (deflated 50%)
  adding: train_Y.npy 	(in=759328) (out=28493) (deflated 96%)
  adding: val_X.npy ....	(in=46160128) (out=23022860) (deflated 50%)
  adding: val_Y.npy	(in=46288) (out=1854) (deflated 96%)
total bytes=806165872, compressed=401941642 -> 50% savings
